# S1-S2 classification (second version)

Original file name: **New_Peakfinder_Zhuoran.ipynb**

Author: Zhuoran Feng

First committed: 18/06/2020

When Joey updated his peak finder, I updated my classification notebook into this one.

In [1]:
import Zip
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time
import math

# Channels in the detector
det_ch = { 'tpc': list(range(0, 247+1)), 'veto': list(range(248, 253+1)), 
          'sum_wv': [254], 'busy_on': [255], 'busy_off': [256],
          'hev_on': [257], 'hev_off': [258], 'muon_veto_trigger': [259], } 
n_channels = 260

# The data files
NG = Zip.ReadZipped("/data/nikhefproject/data/xenon1t/NG") # Neutron data
Rn220 = Zip.ReadZipped("/data/nikhefproject/data/xenon1t/Rn220") # Rn-220 data

In [2]:
# From Joey

def Peaks(pulse):
    
    # The standard variables
    q = 0
    left_t, right_t = 0, 0
    baseline = 16382.
    std = 2.3
    ch = pulse.channel
    wave = pulse.raw_data
    
    # The index for the minimum
    minindex = pulse.right
    
    # Variables for the averages
    ratio = 1
    avfound = 0
    samplelist = []

    s = np.std(wave)
    delta = baseline - np.min(wave)
    if min(s, delta) > 5*std:
        for samples in enumerate(wave):
            base_sub = samples[1] - baseline
            if abs(base_sub) > 5*std:
                q = q + base_sub
             
            # Compute the average of the previous samples
            av = np.average(samplelist)
            
            # left_t is assigned when a significant deviation from the average is recorded
            if abs(samples[1] - av) > 20 and left_t == 0:
                left_t = samples[0]
                avfound = av
             
            # Find the index for the minimum
            if samples[1] == np.min(wave):
                minindex = samples[0]

            # right_t is assigned in a similar way as left_t
            if abs(samples[1] - avfound) < 20 and samples[0] > minindex and right_t == 0: 
                right_t = samples[0]
                
            # Add the current sample to the sample list
            samplelist.append(samples[1])
        ratio = np.min(wave)/int(avfound)
    else:
        baseline = np.average(wave)
        std = s
    width = right_t - left_t 
    if width != 0 and -q/width < 3*std or ratio >= 0.99:
        q = 0
    return (ch, q, width, pulse.left+left_t, ratio)

In [3]:
# A test for channel recording in single event
event = NG.get_single_event(104)

s1_checker = 0
s2_checker = 0

s1_channels = []
s2_channels = []
for p in event.pulses:
    (ch, q, width, t0, ratio) = Peaks(p)
    if q < 0:
        if width < 100:
            if ch not in s1_channels:
                s1_checker += 1
                s1_channels.append(ch)
        else:
            if ch not in s2_channels:
                s2_checker += 1
                s2_channels.append(ch)
        print(ch, ": ", q, width, t0, "| ", ratio)
            

/usr/lib/python3/dist-packages/numpy/lib/function_base.py:1110: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/lib/python3/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


153 :  -92581.0 27 100050 |  0.929258842645
105 :  -90150.0 23 100050 |  0.978376351478
11 :  -45133.0 19 100050 |  0.98193975753
112 :  -84573.0 11 100050 |  0.985625
164 :  -50831.0 23 100051 |  0.9605
165 :  -55266.0 27 100051 |  0.960439972502
13 :  -46121.0 17 100051 |  0.989626296713
95 :  -49849.0 22 100052 |  0.981686355397
141 :  -54073.0 28 100051 |  0.955755530559
151 :  -54081.0 27 100051 |  0.965506467537
76 :  -47396.0 21 100051 |  0.972314230361
204 :  -56210.0 25 100051 |  0.9049375
180 :  -61451.0 32 100051 |  0.91206799575
181 :  -57360.0 26 100051 |  0.9224375
189 :  -60486.0 23 100051 |  0.958877570152
177 :  -55290.0 27 100051 |  0.94725
42 :  -48929.0 20 100051 |  0.946434152135
71 :  -47242.0 19 100051 |  0.986439195101
96 :  -50425.0 26 100051 |  0.96225
110 :  -48204.0 22 100051 |  0.974064120992
122 :  -49999.0 23 100051 |  0.978001374914
46 :  -49423.0 20 100051 |  0.982811425714
159 :  -51959.0 25 100052 |  0.957380327459
149 :  -79572.0 22 100052 |  0.98362

180 :  -40178.0 3 137070 |  0.9885625
13 :  -56914.0 44 137073 |  0.9889375
158 :  -39973.0 3 137116 |  0.9898125
161 :  -59006.0 4 137134 |  0.989750640585
117 :  -40024.0 4 137144 |  0.988813199175
112 :  -326538.0 706 137178 |  0.989935612927
171 :  -47243.0 4 137195 |  0.986813324167
204 :  -73300.0 91 137198 |  0.985813386663
95 :  -71804.0 55 137211 |  0.985188425723
127 :  -39231.0 4 137349 |  0.988875695269
117 :  -40065.0 3 137438 |  0.986625835885
95 :  -65715.0 60 137442 |  0.9824375
181 :  -40299.0 4 137445 |  0.9813125
215 :  -40425.0 4 137457 |  0.98543567946
180 :  -39335.0 3 137570 |  0.98875
170 :  -39959.0 5 137581 |  0.982504373907
132 :  -103813.0 170 137589 |  0.988125
225 :  -55524.0 37 137607 |  0.952633881147
140 :  -53035.0 36 137613 |  0.983626023374
70 :  -58766.0 4 137736 |  0.987249203075
141 :  -39970.0 4 137814 |  0.987251593551
95 :  -103635.0 18 137998 |  0.9828125
112 :  -257471.0 434 138042 |  0.987248406051
84 :  -40163.0 4 138137 |  0.9875
74 :  -83

96 :  -39254.0 3 147126 |  0.989625648397
95 :  -64052.0 41 147162 |  0.9883125
181 :  -40439.0 4 147202 |  0.982623913995
13 :  -87551.0 96 147209 |  0.989251343582
74 :  -72797.0 88 147229 |  0.985124070254
76 :  -39430.0 25 147314 |  0.988686792925
82 :  -48003.0 21 147432 |  0.9836875
10 :  -39948.0 3 147456 |  0.989875632773
158 :  -40138.0 4 147474 |  0.989686855428
74 :  -87934.0 12 147488 |  0.9893125
82 :  -44091.0 14 147816 |  0.9870625
57 :  -40027.0 3 147821 |  0.986813324167
82 :  -52145.0 9 147951 |  0.9866875
170 :  -38905.0 3 147971 |  0.988315421145
151 :  -68105.0 78 147985 |  0.988002249578
105 :  -75467.0 3 148134 |  0.987938253859
180 :  -39561.0 4 148184 |  0.982125
151 :  -40135.0 4 148227 |  0.984501937258
132 :  -40067.0 4 148330 |  0.986250859321
146 :  -42838.0 11 148365 |  0.986875
96 :  -60344.0 18 148471 |  0.983126054622
133 :  -77575.0 4 148505 |  0.985125
95 :  -40264.0 4 148596 |  0.987875
42 :  -54437.0 17 148728 |  0.9891875
60 :  -47131.0 21 149179 

93 :  -49778.0 10 161197 |  0.98925
96 :  -39413.0 3 161206 |  0.988811800738
204 :  -43125.0 11 161219 |  0.98325
60 :  -63272.0 62 161318 |  0.989063867017
30 :  -55164.0 24 161327 |  0.988688206987
151 :  -75842.0 3 161739 |  0.989063867017
94 :  -49328.0 28 161816 |  0.989813136679
215 :  -40220.0 3 161848 |  0.986686667917
116 :  -40013.0 3 161952 |  0.9895625
82 :  -47475.0 20 161964 |  0.9865625
11 :  -44796.0 3 162052 |  0.98875
30 :  -40284.0 5 162065 |  0.9894375
180 :  -40436.0 4 162065 |  0.978375
215 :  -40170.0 4 162098 |  0.988436777299
74 :  -40108.0 3 162132 |  0.988625
146 :  -39487.0 3 162136 |  0.98325
96 :  -39510.0 3 162200 |  0.98806175386
95 :  -41068.0 4 162253 |  0.98525
132 :  -40026.0 3 162292 |  0.9881875
76 :  -73173.0 61 162298 |  0.986750828073
159 :  -41753.0 3 162329 |  0.989250671833
112 :  -44807.0 3 162398 |  0.989936871054
105 :  -76863.0 85 162586 |  0.981813636648
104 :  -56511.0 13 162661 |  0.986250859321
132 :  -40014.0 3 162918 |  0.989813136

...

97 :  -69841.0 4 157217 |  0.983891108891
147 :  -97655.0 18 157225 |  0.989375
186 :  -40206.0 4 157242 |  0.982876070246
229 :  -56024.0 4 157315 |  0.989252015247
78 :  -38913.0 3 157515 |  0.987628092977
193 :  -46270.0 20 157615 |  0.989563804524
47 :  -39995.0 3 157716 |  0.986439195101
78 :  -39041.0 3 157725 |  0.987690577356
193 :  -39324.0 4 157739 |  0.98606424197
187 :  -51016.0 38 158116 |  0.988008993255
75 :  -50285.0 31 158116 |  0.988938191363
229 :  -78434.0 4 158150 |  0.984125992125
210 :  -40981.0 3 158241 |  0.988874304644
238 :  -38165.0 3 158259 |  0.988133158454
210 :  -40222.0 3 158433 |  0.9855615976
187 :  -38145.0 3 158450 |  0.986510117412
211 :  -40233.0 4 158555 |  0.98925
187 :  -38892.0 3 158585 |  0.987697495785
169 :  -41831.0 5 158595 |  0.979938753828
147 :  -39905.0 4 158605 |  0.989875632773
221 :  -40212.0 3 158625 |  0.987125
175 :  -40197.0 3 158683 |  0.987250796825
229 :  -39893.0 4 158712 |  0.988876390451
97 :  -68646.0 80 158766 |  0.9893

136 :  -40103.0 3 175578 |  0.9884375
64 :  -49043.0 13 175689 |  0.9883125
231 :  -61653.0 23 175690 |  0.987564054493
218 :  -40194.0 4 175740 |  0.986188363227
242 :  -42885.0 3 175750 |  0.984688456971
192 :  -38979.0 3 175833 |  0.98300531084
245 :  -40018.0 3 175893 |  0.984689413823
114 :  -47980.0 4 175947 |  0.987375789013
186 :  -40114.0 3 176024 |  0.98825
78 :  -114458.0 86 176066 |  0.978878960195
97 :  -60669.0 3 176076 |  0.984265734266
223 :  -45113.0 6 176427 |  0.982502187227
169 :  -39957.0 3 176470 |  0.989125679645
247 :  -60818.0 60 176484 |  0.988818090955
113 :  -40251.0 8 176495 |  0.98982140627
199 :  -40059.0 3 176605 |  0.987751531059
175 :  -40115.0 4 176616 |  0.985939257593
97 :  -98044.0 159 176626 |  0.987262737263
138 :  -40064.0 3 176692 |  0.986188363227
222 :  -38786.0 4 176724 |  0.989446734108
185 :  -39421.0 4 177109 |  0.9861875
75 :  -40184.0 4 177347 |  0.982439695038
222 :  -37633.0 3 178024 |  0.989386277081
218 :  -65416.0 69 178262 |  0.98

223 :  -39113.0 3 282259 |  0.989688788901
243 :  -58168.0 4 290983 |  0.985452047952
219 :  -41884.0 8 290985 |  0.984252952571
238 :  -38514.0 3 294004 |  0.987943528236
209 :  -39025.0 4 302358 |  0.985198601049
121 :  -46712.0 18 100051 |  0.968316460442
4 :  -48305.0 19 100051 |  0.980256169947
69 :  -47923.0 22 100051 |  0.987250796825
53 :  -47866.0 13 100051 |  0.985563402287
228 :  -54266.0 27 100051 |  0.9635625
38 :  -47813.0 11 100052 |  0.988188238235
92 :  -48246.0 20 100052 |  0.965443979254
67 :  -45192.0 14 100052 |  0.987126609174
81 :  -48638.0 21 100052 |  0.969074097214
66 :  -43961.0 7 100052 |  0.988123515439
40 :  -47519.0 20 100052 |  0.983316670832
80 :  -90688.0 23 100052 |  0.9775
126 :  -52711.0 22 100052 |  0.973375
32 :  -44126.0 4 100052 |  0.987438285107
63 :  -44311.0 4 100052 |  0.984625960877
216 :  -50695.0 26 100052 |  0.95325
217 :  -52650.0 25 100052 |  0.9630625
196 :  -50678.0 27 100052 |  0.976813949128
68 :  -46172.0 15 100053 |  0.9785053736

81 :  -39690.0 4 144508 |  0.987691346454
121 :  -39872.0 3 144767 |  0.988376452943
120 :  -50792.0 31 144985 |  0.988000749953
67 :  -39842.0 3 145303 |  0.989938757655
6 :  -39794.0 3 145713 |  0.989189526964
68 :  -55265.0 44 145837 |  0.983754061485
106 :  -40108.0 3 146090 |  0.988375
41 :  -45634.0 17 146586 |  0.988375726517
217 :  -62158.0 4 146587 |  0.987375789013
108 :  -39750.0 3 146651 |  0.987689808161
54 :  -57945.0 48 146835 |  0.985625898381
41 :  -49335.0 28 146928 |  0.989438160115
22 :  -39956.0 3 147478 |  0.988313230423
228 :  -57021.0 4 147577 |  0.989438160115
121 :  -39867.0 3 147603 |  0.989813773278
68 :  -39893.0 4 147611 |  0.984627882272
126 :  -40914.0 4 147619 |  0.985375914005
63 :  -40200.0 3 148313 |  0.987936746047
101 :  -56657.0 34 149165 |  0.985624101506
92 :  -46307.0 19 149199 |  0.988564644129
54 :  -43400.0 14 149276 |  0.988625710893
81 :  -43074.0 3 149307 |  0.984567322712
121 :  -65947.0 3 149520 |  0.987625773389
126 :  -46215.0 10 1497

126 :  -48852.0 4 179698 |  0.984625
121 :  -41726.0 7 179715 |  0.988814597263
92 :  -42376.0 3 181347 |  0.984502905705
126 :  -46914.0 12 181349 |  0.9855615976
121 :  -51808.0 8 181353 |  0.982189726284
92 :  -39758.0 3 181773 |  0.989252015247
108 :  -40002.0 3 181778 |  0.988313230423
56 :  -64175.0 3 181843 |  0.988316151203
24 :  -41143.0 4 181844 |  0.988124257766
22 :  -43181.0 4 181882 |  0.98850143732
106 :  -96346.0 95 182004 |  0.985063433535
108 :  -45228.0 5 182016 |  0.9858125
54 :  -63674.0 20 182071 |  0.986688331979
216 :  -39436.0 4 182107 |  0.988874304644
92 :  -43069.0 12 182198 |  0.988877085546
120 :  -39770.0 3 182211 |  0.989876265467
92 :  -48099.0 5 182463 |  0.982816795801
68 :  -38973.0 2 182705 |  0.989563804524
101 :  -41107.0 4 182922 |  0.985
56 :  -72018.0 35 182926 |  0.976569821931
21 :  -36661.0 3 183015 |  0.98876334353
23 :  -37898.0 2 183425 |  0.989695872104
81 :  -45374.0 7 183438 |  0.98931650631
121 :  -42588.0 4 183487 |  0.987625773389
8

In [4]:
print('s1_channels: ' +  str(s1_checker))
s1_channels.sort()
print(s1_channels)
print('s2_channels: ' +  str(s2_checker))
s2_channels.sort()
print(s2_channels)

s1_channels: 200
[4, 6, 7, 9, 10, 11, 13, 15, 16, 17, 19, 21, 22, 23, 24, 28, 30, 32, 33, 35, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 63, 64, 66, 67, 68, 69, 70, 71, 72, 74, 75, 76, 77, 78, 80, 81, 82, 83, 84, 85, 87, 89, 90, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 119, 120, 121, 122, 123, 125, 126, 127, 128, 129, 131, 132, 133, 136, 138, 140, 141, 142, 143, 144, 145, 146, 147, 149, 151, 153, 154, 155, 156, 157, 158, 159, 160, 161, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 179, 180, 181, 182, 184, 185, 186, 187, 188, 189, 191, 192, 193, 194, 195, 196, 197, 199, 200, 201, 202, 204, 205, 207, 208, 209, 210, 211, 212, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 235, 236, 237, 238, 240, 241, 242, 243, 245, 246, 247, 254]
s2_channels: 216
[0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15

Event 103:

8 :  -183503.0 163 100050 |  0.948378226361
11 :  -241537.0 185 100050 |  0.883577052868
42 :  -141786.0 106 100050 |  0.955377788888
43 :  -304823.0 167 100050 |  0.896125
45 :  -276889.0 208 100050 |  0.864445972127
71 :  -177320.0 99 100050 |  0.981064866892
10 :  -490027.0 237 100050 |  0.626898318855
9 :  -632281.0 222 100051 |  0.689644397225
44 :  -414843.0 232 100051 |  0.685976752906
72 :  -150669.0 122 100051 |  0.965706789931

These peaks are obviously the s1 peak (t0 ~ 100050), but they are identified as s2

so I decided to use width and the position to identify s1

## Classification also use S1 position

In [5]:
# A test for single event

event = NG.get_single_event(104)
s1_channellist = []
s2_channellist = []

s1_checker = 0
s2_checker = 0

s1_q = np.zeros(260)
s1_t0 = np.zeros(260)
s2_q = np.zeros(260)
s2_t0 = np.zeros(260)

for p in event.pulses:
    (ch, q, width, t0, ratio) = Peaks(p)
    if q < 0:
        if width < 100 or (t0>100000 and t0<100100):
            if ch in det_ch['tpc'] and ch not in s1_channellist:
                s1_channellist.append(ch)
                s1_checker += 1
            
            s1_q[ch] = -q
            s1_t0[ch] = t0
        else:
            if s2_q[ch] == 0:
                s2_t0[ch] = t0
            if ch in det_ch['tpc'] and ch not in s2_channellist:
                s2_channellist.append(ch)
                s2_checker += 1
            s2_q[ch] = s2_q[ch] - q
if s1_checker>2 and s2_checker>3:
    s1_channellist.sort()
    s2_channellist.sort()
    info = {'event': event.event_number, 's1': s1_q[254], 's2': s2_q[254], 
            's1_position': s1_t0[254], 's2_position': s2_t0[254], 
            's1_channels':s1_channellist, 's2_channels':s2_channellist}


print(s1_checker, s2_checker)

/usr/lib/python3/dist-packages/numpy/lib/function_base.py:1110: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/lib/python3/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


199 215


In [6]:
print(info)

{'event': 104, 's1': 68625.0, 's2': 1559629.0, 's1_position': 100055.0, 's2_position': 134048.0, 's1_channels': [4, 6, 7, 9, 10, 11, 13, 15, 16, 17, 19, 21, 22, 23, 24, 28, 30, 32, 33, 35, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 63, 64, 66, 67, 68, 69, 70, 71, 72, 74, 75, 76, 77, 78, 80, 81, 82, 83, 84, 85, 87, 89, 90, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 119, 120, 121, 122, 123, 125, 126, 127, 128, 129, 131, 132, 133, 136, 138, 140, 141, 142, 143, 144, 145, 146, 147, 149, 151, 153, 154, 155, 156, 157, 158, 159, 160, 161, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 179, 180, 181, 182, 184, 185, 186, 187, 188, 189, 191, 192, 193, 194, 195, 196, 197, 199, 200, 201, 202, 204, 205, 207, 208, 209, 210, 211, 212, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 235, 236, 237, 238, 240, 

## Try to record diffrent channels: still slow

In [7]:
info_list = []
width_list = []


for event in NG.get_events():
    #if event.event_number%10 == 0:
    print(event.event_number)
    
    s1_q = np.zeros(260)
    s1_t0 = np.zeros(260)
    s2_q = np.zeros(260)
    s2_t0 = np.zeros(260)
    
    s1_channellist = []
    s2_channellist = []
    
    s1_checker = 0
    s2_checker = 0
    
    for p in event.pulses:
        (ch, q, width, t0, ratio) = Peaks(p)
        if q < 0:
            width_list.append(width)
            if width < 100:
                s1_q[ch] = -q
                s1_t0[ch] = t0
                if ch in det_ch['tpc'] and ch not in s1_channellist:
                    s1_channellist.append(ch)
                    s1_checker += 1
            else:
                if s2_q[ch] == 0:
                    s2_t0[ch] = t0
                if ch in det_ch['tpc'] and ch not in s2_channellist:
                    s2_channellist.append(ch)
                    s2_checker += 1
                s2_q[ch] = s2_q[ch] - q
    if s1_checker>2 and s2_checker>3:
        s1_channellist.sort()
        s2_channellist.sort()
        info = {'event': event.event_number, 's1': s1_q, 's2': s2_q, 
                's1_position': s1_t0, 's2_position': s2_t0, 
                's1_channels':s1_channellist, 's2_channels':s2_channellist}
        info_list.append(info)
    if event.event_number == 10:
        break
    



0


/usr/lib/python3/dist-packages/numpy/lib/function_base.py:1110: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/lib/python3/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1
2
3
4
5
6
7
8
9
10


In [8]:
print(len(info_list))

9


In [9]:
print(info_list[0])

{'event': 0, 's1': array([      0.,       0.,       0.,       0.,   39778.,   40089.,
         39878.,   40038.,       0.,   40131.,   40007.,   40113.,
             0.,   51059.,   39268.,   42917.,   44389.,  219613.,
             0.,   44494.,   41059.,   38458.,   40150.,       0.,
         40959.,   45202.,       0.,       0.,   40193.,   43733.,
         39028.,       0.,   39219.,   42530.,       0.,   40071.,
         45552.,   40128.,   39777.,   46593.,   39905.,   39891.,
         40010.,   85446.,   67698.,       0.,   45228.,   43155.,
        323549.,   42477.,   39281.,   39848.,   54571.,       0.,
         39926.,       0.,   39611.,   40179.,       0.,   48350.,
         53410.,   82629.,       0.,   40131.,   39897.,       0.,
         48450.,   40084.,   43687.,   40184.,   47430.,   40067.,
         51399.,       0.,   42291.,   40087.,   39261.,   53009.,
         61228.,       0.,   59377.,   51169.,   46786.,   53472.,
         40036.,   68740.,       0.,   9525